In [1]:
# import dependencies
import pandas as pd
from config import YT_API
import json
import requests
from googleapiclient.discovery import build
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
#enter video id
video_id="9bZkp7q19f0"

In [10]:
def get_video_metadata(video_id):
    # creating youtube resource object
    youtube = build('youtube', 'v3', developerKey=YT_API)

    # get video title
    video_title = youtube.videos().list(part='snippet', id=video_id).execute()['items'][0]['snippet']['title']
    
    # retrieve youtube video results
    video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()

    # create empty dataframe
    df = pd.DataFrame(columns=['id', 'author', 'thumbsup', 'text','location', '# of replies', 'replies'])

    # iterate video response
    while video_response:

    # export comments to a dataframe
        for i in range(len(video_response['items'])):
            replies = []
            id = video_response['items'][i]['snippet']['topLevelComment']['id']
            author = video_response['items'][i]['snippet']['topLevelComment']['snippet']['authorDisplayName']
            text = video_response['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal']
            location= video_response['items'][i]['statistics']['location']
            #spam= video_response['items'][i]['snippet']['moderationStatus']['likelySpam']
            #views= video_response['items'][i]['statistics']['viewCount']
            #comments= video_response['items'][i]['statistics']['commentCount']
            replycount = video_response['items'][i]['snippet']['totalReplyCount']
            if replycount>0:
                try:
                    for reply in video_response['items'][i]['replies']['comments']:
                        reply = reply['snippet']['textOriginal']
                        replies.append(reply)
                except:
                    pass
            df = df.append({
                'id': id, 
                'author': author, 
                'thumbsup': int(video_response['items'][i]['snippet']['topLevelComment']['snippet']['likeCount']),
                'text': text,
                'location':location, 
                #'class': spam,
                #'views': views,
                #'comment_amount':comments,
                '# of replies': replycount, 
                'replies': replies 
                }, 
                ignore_index=True)
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            pageToken=video_response['nextPageToken']
            ).execute()
        else:
            break

    # change data types
    df['thumbsup'] = df['thumbsup'].astype(int)
    df['# of replies'] = df['# of replies'].astype(int)

    return df, video_title

In [11]:
df, video_title = get_video_metadata(video_id)

KeyError: 'statistics'